In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import polygon
import requests
import re

In [ ]:
map_df = gpd.read_file("Westminster_Parliamentary_Constituencies_December_2017_Generalised_Clipped_Boundaries_in_the_UK")

In [ ]:
map_df.head()

In [ ]:
BASE_URL = 'https://www.bbc.co.uk/news/politics/constituencies/'
voteExtractor = re.compile("ge2019-constituency-result__party-name\">(.*)<[\S\s]*?ge2019-constituency-result__details-value\">(.*)<")

In [ ]:
import pickle
constitData = pickle.load(open("constitData.p", "rb"))
constitData

In [ ]:
constitData = []
for const in map_df['pcon17cd']:
    responce = requests.get(BASE_URL + const)
    text = responce.text
    votes = voteExtractor.findall(text)
    constitData.append(votes)
    print(map_df[map_df["pcon17cd"] == const]['pcon17nm'],":", votes)

In [ ]:
pickle.dump(constitData, open("constitData.p","wb"))

In [ ]:
map_df["votes"] = constitData

In [ ]:
df = pd.DataFrame()

In [ ]:
parties = dict()
for result in constitData:
    for party in result:
        if party[0] not in parties:
            parties[party[0]] = 0
        parties[party[0]] += int(party[1].replace(",",""))

In [ ]:
parties.keys()

In [ ]:
plt.bar(list(parties.keys())[:5], list(parties.values())[:5])

In [ ]:
plt.bar(["Con + Brexit", "Lab + LD + SNP"], 
       [parties["Conservative"] + parties["The Brexit Party"],parties["Labour"] + parties["Liberal Democrat"] + parties["Scottish National Party"]] )

In [ ]:
parties["Labour"] + parties["Liberal Democrat"] 

In [ ]:
parties["Conservative"]

In [ ]:
c_dict = {
    "Conservative": "#0087DC",
    "Labour" : "#DC241f",
    "Liberal Democrat": "#FDBB30",
    "Scottish National Party" : "#FFFF00",
    "Green": "#528D6B",
    "The Brexit Party": '#12B6CF',
    "Democratic Unionist Party": "#D46A4C",
}

In [ ]:
colours = []
for party in parties.keys():
    if party in c_dict.keys():
        colours.append(c_dict[party])
    else:
        colours.append("#DDDDDD")

In [ ]:
partySorts = list(zip(parties.keys(),parties.values()))

In [ ]:
def sortSecond(val): 
    return val[1] 

partySorts.sort(key=sortSecond, reverse=True)

In [ ]:
colours = []
for party in partySorts:
    if party[0] in c_dict.keys():
        colours.append(c_dict[party[0]])
    else:
        colours.append("#DDDDDD")

In [ ]:
partiesList,values = map(list,zip(*partySorts))

In [ ]:
noParties = 7
lessLabels = partiesList[:noParties]
lessLabels.extend([""]*(len(parties)-(noParties+1)))
lessLabels.append("Other")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.set_facecolor('azure')
fig.suptitle("First Past the Post")

plt.subplot(1,2,2)
plt.title("Vote Share")
plt.pie(values, labels=lessLabels, colors=colours)

ax = plt.subplot(1,2,1)
plt.title("Total Votes")
barlist = plt.bar(["Con + Brexit", "Lab + LD + SNP"], 
       [parties["Conservative"] + parties["The Brexit Party"],parties["Labour"] + parties["Liberal Democrat"] + parties["Scottish National Party"]])
barlist[0].set_color("#0087DC")
barlist[1].set_color("#DC241f")
plt.ticklabel_format(axis="y", style="plain")


plt.show()